In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../')
from cleaner import Cleaner, Corretor
from tqdm.auto import tqdm

In [2]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold.csv')
dados

FileNotFoundError: [Errno 2] No such file or directory: '/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold.csv'

In [ ]:
dados.columns

In [ ]:
print(dados['certidao_protesto'].value_counts())
print(dados['certificado_boas_praticas'].value_counts())
print(dados['comprovante_localizacao'].value_counts())
print(dados['idoneidade_financeira'].value_counts())
print(dados['integralizado'].value_counts())
print(dados['licenca_ambiental'].value_counts())
print(dados['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
dados.drop_duplicates(subset=['text'], inplace=True)
dados

In [ ]:
dados.reset_index(drop=True, inplace=True)
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold.csv', index=False)

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(dados.values, dados[dados.columns[1:]].values, test_size = 0.20)
X_test, y_test, X_val, y_val = iterative_train_test_split(X_test, y_test, test_size = 0.5)

In [ ]:
treino, teste = train_test_split(dados, test_size=0.20)
teste, validacao = train_test_split(teste, test_size=0.50)

In [ ]:
treino = pd.DataFrame(X_train, columns=dados.columns)
teste = pd.DataFrame(X_test, columns=dados.columns)
validacao = pd.DataFrame(X_val, columns=dados.columns)

In [ ]:
treino.reset_index(drop=True, inplace=True)
treino

In [ ]:
teste.reset_index(drop=True, inplace=True)
teste

In [ ]:
validacao.reset_index(drop=True, inplace=True)
validacao

In [ ]:
treino.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_treino.csv', index=False)
teste.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_teste.csv', index=False)
validacao.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_validacao.csv', index=False)

### Modificação da base de dados

In [ ]:
import pandas as pd
import re

In [ ]:
treino = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_treino.csv')
teste = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_teste.csv')
validacao = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_validacao.csv')

In [ ]:
teste = pd.concat([teste, validacao], ignore_index=True)
teste

In [ ]:
print(treino['certidao_protesto'].value_counts())
print(treino['certificado_boas_praticas'].value_counts())
print(treino['comprovante_localizacao'].value_counts())
print(treino['idoneidade_financeira'].value_counts())
print(treino['integralizado'].value_counts())
print(treino['licenca_ambiental'].value_counts())
print(treino['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
print(teste['certidao_protesto'].value_counts())
print(teste['certificado_boas_praticas'].value_counts())
print(teste['comprovante_localizacao'].value_counts())
print(teste['idoneidade_financeira'].value_counts())
print(teste['integralizado'].value_counts())
print(teste['licenca_ambiental'].value_counts())
print(teste['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
len(treino[treino['n_min_max_limitacao_atestados'] == 1])

In [ ]:
import re
n_min_max_limitacao_atestados = re.compile(r'((((dois|duas)|(tr[êe]s)|(quatro)|(cinco)).{0,10}((atestado[s]?)|(certid[aãoões]*))).{0,20}((capacidade t[eé]cnica)|(qualifica[cç][aã]o t[eé]cnica)))', flags=re.IGNORECASE|re.S)
n_min_max_limitacao_atestados_numero = re.compile(r'(((<numero>).{0,10}((atestado[s]?)|(certid[aãoões]*))).{0,20}((capacidade t[eé]cnica)|(qualifica[cç][aã]o t[eé]cnica)))', flags=re.IGNORECASE|re.S)

In [ ]:
import random
lista_qtds = ['dois', 'tres', 'quatro', 'cinco']

In [ ]:
def substituir_numero(match):
    numero_extenso = random.choice(['dois', 'tres', 'quatro', 'cinco'])
    return match.group(0).replace('<numero>', numero_extenso)

In [ ]:
def ajustaNumero(treino):
    for indice in treino.index:
        if treino['n_min_max_limitacao_atestados'][indice] == 1:
            busca = re.search(n_min_max_limitacao_atestados_numero, treino['text'][indice])
            if busca != None:
                texto_substituido = re.sub(n_min_max_limitacao_atestados_numero, substituir_numero, treino['text'][indice])
                print(busca)
                treino.at[indice, 'text'] = texto_substituido
    return treino

In [ ]:
def ajustaNumeroNegativo(treino):
    for indice in treino.index:
        if treino['n_min_max_limitacao_atestados'][indice] == 0:
            busca = re.search(n_min_max_limitacao_atestados_numero, treino['text'][indice])
            if busca != None:
                #texto_substituido = re.sub(n_min_max_limitacao_atestados_numero, substituir_numero, treino['text'][indice])
                print(busca)
                #treino.at[indice, 'text'] = texto_substituido
    #return treino

In [ ]:
ajustaNumeroNegativo(treino)

In [ ]:
treino = ajustaNumero(treino)
teste = ajustaNumero(teste)
validacao = ajustaNumero(validacao)

In [ ]:
treino.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_treino_ajustado.csv', index=False)
teste.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_teste_ajustado.csv', index=False)
validacao.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_bid_notices_gold_validacao_ajustado.csv', index=False)

### Divisão stratified

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(dados.values, dados[dados.columns[1:]].values, test_size = 0.25)
#X_test, y_test, X_val, y_val = iterative_train_test_split(X_test, y_test, test_size = 0.5)

In [ ]:
treino = pd.DataFrame(X_train, columns=dados.columns)
teste = pd.DataFrame(X_test, columns=dados.columns)
#validacao = pd.DataFrame(X_val, columns=dados.columns)

In [ ]:
print(treino['certidao_protesto'].value_counts())
print(treino['certificado_boas_praticas'].value_counts())
print(treino['comprovante_localizacao'].value_counts())
print(treino['idoneidade_financeira'].value_counts())
print(treino['integralizado'].value_counts())
print(treino['licenca_ambiental'].value_counts())
print(treino['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
print(teste['certidao_protesto'].value_counts())
print(teste['certificado_boas_praticas'].value_counts())
print(teste['comprovante_localizacao'].value_counts())
print(teste['idoneidade_financeira'].value_counts())
print(teste['integralizado'].value_counts())
print(teste['licenca_ambiental'].value_counts())
print(teste['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
treino.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/Stratified/dataset_bid_notices_gold_treino_stratified.csv', index=False)
teste.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/Stratified/dataset_bid_notices_gold_teste_stratified.csv', index=False)